# Flyweight Pattern

- __Type:__ Structural
- __Popularity: ★★★☆☆__
- __Complexity: ★★★☆☆__

## Intent

- Minimize memory usage by sharing common parts of state between objects
- Support large numbers of similar objects efficiently
- Separate intrinsic (shared) from extrinsic (unique) state
- Maintain performance when object quantity increases dramatically

## Problem

When applications need to create a large number of similar objects, memory consumption can become a critical issue. Creating thousands or millions of individual objects with redundant data can quickly exhaust available memory and degrade performance.

For example:

- A text editor needs to represent each character with its formatting (font, size, style)
- A game world contains thousands of similar graphical elements (trees, particles)
- A simulation tracks millions of particles that share many common properties

Without a proper pattern to handle this scenario:
- Memory usage grows linearly with the number of objects
- Identical data is duplicated across many objects
- Performance suffers due to excessive memory allocation and garbage collection

## Solution

The Flyweight pattern solves this problem by sharing common parts of state between multiple objects instead of keeping all of the data in each object. The pattern separates object state into two parts:

- **Intrinsic state**: Data that can be shared across multiple objects (immutable, context-independent)
- **Extrinsic state**: Data that is unique to each object instance (mutable, context-specific)

Key components:

1. **Flyweight**: Contains the intrinsic (shared) state that can be used across multiple contexts
2. **FlyweightFactory**: Creates and manages flyweight objects, ensuring that flyweights are shared properly
3. **Context**: Contains extrinsic state and references to flyweight objects
4. **Client**: Computes or stores the extrinsic state and passes it to the flyweight when needed

The pattern typically uses a factory to maintain a pool of existing flyweights, so they can be reused rather than creating new objects. When a client requests a flyweight, the factory either returns an existing instance or creates a new one if necessary.

## Diagram

```mermaid
classDiagram
    class Flyweight {
        -shared_state: Any
        +operation(unique_state)
    }
    
    class FlyweightFactory {
        -flyweights: Dict
        +get_flyweight(shared_state): Flyweight
        -get_key(state): String
    }
    
    class Client {
        +method()
    }
    
    Client --> FlyweightFactory: uses
    FlyweightFactory --> Flyweight: creates/manages
    Client --> Flyweight: stores reference
```

## Implementation 1: Basic Flyweight Pattern

In [1]:
from typing import Dict


# Flyweight class - contains intrinsic state shared across objects
class Flyweight:
    def __init__(self, shared_state):
        self._shared_state = shared_state

    def operation(self, unique_state):
        """Operation using both shared (intrinsic) and unique (extrinsic) state"""
        shared = ", ".join(str(item) for item in self._shared_state)
        unique = ", ".join(str(item) for item in unique_state)
        return f"Flyweight: Displaying shared state ({shared}) and unique state ({unique})"


# Factory that creates and manages flyweight objects
class FlyweightFactory:
    _flyweights: Dict[str, Flyweight] = {}

    def __init__(self, initial_flyweights):
        # Initialize with a set of flyweights
        for state in initial_flyweights:
            self.get_flyweight(state)

    def get_flyweight(self, shared_state) -> Flyweight:
        """Returns an existing flyweight with the specified shared state or creates a new one"""
        # Convert shared state into a string key
        key = self._get_key(shared_state)

        # If this flyweight doesn't exist, create it
        if key not in self._flyweights:
            print(f"FlyweightFactory: Creating new flyweight with state {shared_state}")
            self._flyweights[key] = Flyweight(shared_state)
        else:
            print(f"FlyweightFactory: Reusing existing flyweight with state {shared_state}")

        return self._flyweights[key]

    def list_flyweights(self):
        """Lists all flyweights and their keys"""
        count = len(self._flyweights)
        print(f"\nFlyweightFactory: I have {count} flyweights:")
        for key in self._flyweights.keys():
            print(key)

    def _get_key(self, state) -> str:
        """Create a hash key from shared state"""
        # Sort state to ensure consistent key generation
        return "_".join(sorted(str(item) for item in state))

### Usage

In [2]:
def add_car_to_database(factory, license_plate, owner, brand, model, color):
    print("\nClient: Adding a car to database.")

    # Shared (intrinsic) state: brand, model, color
    flyweight = factory.get_flyweight([brand, model, color])

    # Unique (extrinsic) state: license plate, owner
    result = flyweight.operation([license_plate, owner])
    print(result)


if __name__ == "__main__":
    # Create factory with initial flyweights
    factory = FlyweightFactory(
        [
            ["Chevrolet", "Camaro", "red"],
            ["Mercedes Benz", "C300", "black"],
            ["Mercedes Benz", "C500", "red"],
            ["BMW", "M5", "red"],
            ["BMW", "X6", "white"],
        ]
    )

    factory.list_flyweights()

    # Add cars using the factory - notice reuse of flyweights
    add_car_to_database(factory, "CL234IR", "James Doe", "BMW", "M5", "red")
    add_car_to_database(factory, "CL567IR", "Alice Smith", "BMW", "X1", "blue")

    factory.list_flyweights()

FlyweightFactory: Creating new flyweight with state ['Chevrolet', 'Camaro', 'red']
FlyweightFactory: Creating new flyweight with state ['Mercedes Benz', 'C300', 'black']
FlyweightFactory: Creating new flyweight with state ['Mercedes Benz', 'C500', 'red']
FlyweightFactory: Creating new flyweight with state ['BMW', 'M5', 'red']
FlyweightFactory: Creating new flyweight with state ['BMW', 'X6', 'white']

FlyweightFactory: I have 5 flyweights:
Camaro_Chevrolet_red
C300_Mercedes Benz_black
C500_Mercedes Benz_red
BMW_M5_red
BMW_X6_white

Client: Adding a car to database.
FlyweightFactory: Reusing existing flyweight with state ['BMW', 'M5', 'red']
Flyweight: Displaying shared state (BMW, M5, red) and unique state (CL234IR, James Doe)

Client: Adding a car to database.
FlyweightFactory: Creating new flyweight with state ['BMW', 'X1', 'blue']
Flyweight: Displaying shared state (BMW, X1, blue) and unique state (CL567IR, Alice Smith)

FlyweightFactory: I have 6 flyweights:
Camaro_Chevrolet_red
C30

## Implementation 2: Text Formatter Example

In [3]:
class CharacterStyle:
    """Flyweight - stores character formatting (shared state)"""

    def __init__(self, font_name, font_size, bold=False, italic=False):
        self.font_name = font_name
        self.font_size = font_size
        self.bold = bold
        self.italic = italic

    def __str__(self):
        style = []
        style.append(f"Font: {self.font_name}")
        style.append(f"Size: {self.font_size}")
        if self.bold:
            style.append("Bold")
        if self.italic:
            style.append("Italic")
        return ", ".join(style)


class StyleFactory:
    """Factory that creates and manages character style flyweights"""

    _styles = {}

    @classmethod
    def get_style(cls, font_name, font_size, bold=False, italic=False):
        # Create a key for this style combination
        key = (font_name, font_size, bold, italic)

        # Return existing style or create a new one
        if key not in cls._styles:
            cls._styles[key] = CharacterStyle(font_name, font_size, bold, italic)
            print(f"Created new style: {cls._styles[key]}")
        else:
            print(f"Reusing style: {cls._styles[key]}")

        return cls._styles[key]

    @classmethod
    def get_style_count(cls):
        return len(cls._styles)


class Character:
    """Context class that uses flyweight for styling"""

    def __init__(self, char, style):
        self.char = char  # Extrinsic state (unique per character)
        self.style = style  # Intrinsic state (shared style)

    def __str__(self):
        return f"{self.char} ({self.style})"


class TextEditor:
    def __init__(self):
        self.characters = []

    def add_character(self, char, font_name, font_size, bold=False, italic=False):
        style = StyleFactory.get_style(font_name, font_size, bold, italic)
        self.characters.append(Character(char, style))

    def __str__(self):
        return "".join(char.char for char in self.characters)

### Usage

In [4]:
if __name__ == "__main__":
    editor = TextEditor()

    # Add text with various styles
    editor.add_character("H", "Arial", 12, True, False)
    editor.add_character("e", "Arial", 12, True, False)
    editor.add_character("l", "Arial", 12, True, False)
    editor.add_character("l", "Arial", 12, True, False)  # Reuse existing style
    editor.add_character("o", "Arial", 12, True, False)
    editor.add_character(" ", "Arial", 12, False, False)
    editor.add_character("W", "Times New Roman", 14, False, True)
    editor.add_character("o", "Times New Roman", 14, False, True)
    editor.add_character("r", "Times New Roman", 14, False, True)
    editor.add_character("l", "Times New Roman", 14, False, True)
    editor.add_character("d", "Times New Roman", 14, False, True)
    editor.add_character("!", "Arial", 12, True, False)

    print(f"\nText: {editor}")
    print(f"Total characters: {len(editor.characters)}")
    print(f"Total styles (flyweights): {StyleFactory.get_style_count()}")

Created new style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Reusing style: Font: Arial, Size: 12, Bold
Created new style: Font: Arial, Size: 12
Created new style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Times New Roman, Size: 14, Italic
Reusing style: Font: Arial, Size: 12, Bold

Text: Hello World!
Total characters: 12
Total styles (flyweights): 3


## Real-world analogies

1. **Shared Office Equipment:**
   - Instead of providing each employee with their own printer, scanner, and copier, an office typically has shared equipment used by everyone.
   - The equipment (printer) represents the flyweight with shared state (model, capabilities).
   - The print jobs represent the extrinsic state (unique content, settings).

2. **Car Factory and Car Models:**
   - A car manufacturer designs a model once (blueprints, specifications) but produces thousands of individual cars.
   - The model design is the flyweight (shared state).
   - Each specific car has unique extrinsic state (VIN number, color variations, features).

3. **Library Books:**
   - A library has one copy of a book title that's borrowed by many readers over time.
   - The book's content is the immutable shared state (flyweight).
   - Who has borrowed it, due dates, and wear condition are extrinsic state.

## When to use

- When an application needs to create a large number of similar objects
- When memory usage is a critical concern in your application
- When objects contain state that can be separated into intrinsic and extrinsic parts
- When many objects can share a common state that doesn't change frequently or at all
- When the identity of objects doesn't matter to the application

The Flyweight pattern is particularly valuable in applications that handle:
- Text processing and formatting
- Graphics rendering with many similar elements
- Game development with repeated game objects
- Network applications managing many connections with shared configurations
- Caching systems that require efficient memory utilization

## Python-specific implementation notes

- Python's built-in string interning automatically applies the flyweight concept for strings, improving memory efficiency
- The `functools.lru_cache` decorator can be used to implement simple flyweight caching for function return values
- Python dictionaries are often used to implement flyweight factories efficiently
- The `__slots__` attribute can be used in flyweight classes to further reduce memory consumption
- Python's weak references (`weakref` module) can help manage flyweight objects that are no longer needed

Example of using `__slots__` to optimize memory usage in flyweights:
```python
class OptimizedFlyweight:
    __slots__ = ['_shared_state']
    
    def __init__(self, shared_state):
        self._shared_state = shared_state
```

Example using `functools.lru_cache` as a simple flyweight factory:
```python
import functools

@functools.lru_cache(maxsize=100)
def get_flyweight(shared_state):
    return Flyweight(shared_state)
```

## Related patterns

- **Singleton**: Both patterns can be used to share common resources, but Singleton ensures only one instance exists while Flyweight allows multiple shared instances with different intrinsic states

- **Composite**: Flyweights can be used as leaf nodes in a Composite pattern to save memory when the tree structure has many similar leaf objects

- **State/Strategy**: These patterns can use Flyweight to share common state objects among multiple contexts

- **Factory Method**: Often used with Flyweight to create and manage flyweight objects

- **Object Pool**: Both patterns aim to reuse objects, but the Object Pool pattern focuses on reusing complete objects that are expensive to create, while Flyweight focuses on sharing parts of object state